# AI Training
## Edge model training

In [1]:
!python3 -m pip install tensorflow
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler


You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3 -m pip install --upgrade pip' command.


In [13]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import Input, Model, layers
from sklearn.model_selection import train_test_split

# Load and preprocess data
RAIN_THRESHHOLD = 0.5
data = pd.read_csv('weather_data.csv', low_memory=False)
data = data.sample(frac=1, random_state=2).reset_index(drop=True)
data = data.replace('', float('nan')).dropna()
data = data.replace(' ', float('nan'))
data = data.dropna(subset=['wetb', 'vappr', 'rhum', 'vis'])
X = data.drop(columns=['rain','date'])
y = data['rain']
print(data.isna().sum())

# Ensure all data is numeric
X = X.astype(float)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape y to (num_samples, 1) for regression
y_train = y_train.values.reshape(-1, 1)
y_test = y_test.values.reshape(-1, 1)

# Define the model
inputs = Input(shape=(X_train.shape[1],))
x = layers.Dense(64, activation='relu', name='e1')(inputs)
x_edge = layers.Dense(32, activation='relu', name='e2')(x)
edge_outputs = layers.Dense(1, activation='linear', name='edge_output')(x_edge)
x = layers.Dense(64, activation='relu', name='controlmodel')(x_edge)
x = layers.Dense(128, activation='relu', name='l1')(x)
x = layers.Dense(256, activation='relu', name='l2')(x)
x = layers.Dense(128, activation='relu', name='l3')(x)
x = layers.Dense(64, activation='relu', name='l4')(x)
x = layers.Dense(64, activation='relu', name='l5')(x)
control_outputs = layers.Dense(1, activation='linear', name='control_output')(x)

# Create and compile the combined model
combined_model = Model(inputs=inputs, outputs=[edge_outputs, control_outputs])
combined_model.compile(optimizer='adam',
                       loss={'edge_output': 'mean_squared_error',
                             'control_output': 'mean_squared_error'},
                       metrics={'edge_output': 'mae',
                                'control_output': 'mae'})

# Train the combined model
combined_model.fit(X_train, [y_train, y_train], 
                   epochs=30, batch_size=32, 
                   validation_data=(X_test, [y_test, y_test]))

# Define the edge_model
edge_model = Model(inputs=inputs, outputs=[combined_model.get_layer('e2').output, edge_outputs])

# x§This needs training 
# Set last as untrainable, then train last ones, this is not needed for contorl model becasue its last output is trained when entire model 
# gets trained
for layer in edge_model.layers:
    layer.trainable = False

edge_model.get_layer('edge_output').trainable= True

edge_model.compile(optimizer='adam',
                   loss={'e2': None, 'edge_output': 'mean_squared_error'},
                   metrics={'edge_output': 'mae'})

history = edge_model.fit(X_train, [y_train, y_train], 
               epochs=30, batch_size=32, 
               validation_data=(X_test, [y_test, y_test]))

# Define the control_model
control_model_input = Input(shape=(32,))  # Shape of the intermediate layer output
z = combined_model.get_layer('controlmodel')(control_model_input)
z = combined_model.get_layer('l1')(z)
z = combined_model.get_layer('l2')(z)
z = combined_model.get_layer('l3')(z)
z = combined_model.get_layer('l4')(z)
z = combined_model.get_layer('l5')(z)

control_model_outputs = combined_model.get_layer('control_output')(z)
control_model = Model(inputs=control_model_input, outputs=control_model_outputs)

# Inspect the models
edge_model.summary()
control_model.summary()


date     0
ind      0
rain     0
ind1     0
temp     0
ind2     0
wetb     0
dewpt    0
vappr    0
rhum     0
msl      0
ind3     0
wdsp     0
ind4     0
wddir    0
ww       0
w        0
sun      0
vis      0
clht     0
clamt    0
dtype: int64
Epoch 1/30
3184/3184 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - control_output_loss: 160.9835 - control_output_mae: 2.3720 - edge_output_loss: 14000.2979 - edge_output_mae: 34.2581 - loss: 14161.2793 - val_control_output_loss: 1.4006 - val_control_output_mae: 1.0015 - val_edge_output_loss: 88.8663 - val_edge_output_mae: 8.0245 - val_loss: 90.2720
Epoch 2/30
3184/3184 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - control_output_loss: 1.0781 - control_output_mae: 0.5726 - edge_output_loss: 1726.9899 - edge_output_mae: 11.6927 - loss: 1728.0688 - val_control_output_loss: 0.4656 - val_control_output_mae: 0.4943 - val_edge_output_loss: 3.3812 - val_edge_output_mae: 1.4594 - val_loss: 3.8471
Epoch 3/30
3184/3184 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - control_output_loss: 13.

Model: "functional_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)      │ (None, 19)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ e1 (Dense)                      │ (None, 64)             │         1,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ e2 (Dense)                      │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ edge_output (Dense)             │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,461 (13.52 KB)

 Trainable params: 33 (132.00 B)

...

Model: "functional_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_9 (InputLayer)      │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ controlmodel (Dense)            │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ l1 (Dense)                      │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ l2 (Dense)                      │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ l3 (Dense)                      │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ l4 (Dense)                      │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ l5 (Dense)                      │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ control_output (Dense)          │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 88,833 (347.00 KB)

 Trainable params: 88,833 (347.00 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:

results = combined_model.evaluate(X_test, [y_test, y_test])
print("Test Loss:", results[0])
print("Edge Output - MAE:", results[1])
print("Edge Output - MSE - How much its off by on average:", results[2])
print("Control Output - MAE:", results[3])
print("Control Output - MSE:", results[4])

796/796 ━━━━━━━━━━━━━━━━━━━━ 0s 573us/step - control_output_loss: 0.1849 - control_output_mae: 0.1578 - edge_output_loss: 0.3360 - edge_output_mae: 0.4254 - loss: 0.5208
Test Loss: 0.541490375995636
Edge Output - MAE: 0.34628161787986755
Edge Output - MSE - How much its off by on average: 0.19510380923748016
Control Output - MAE: 0.15660102665424347
Control Output - MSE: 0.42397430539131165


In [15]:

edge_model.export("expt")
converter = tf.lite.TFLiteConverter.from_saved_model('./expt')
tflite_model = converter.convert()
with open('../deployment/models/trained_edge_model.tflite', 'wb') as f:
  f.write(tflite_model)

edge_model.save('../deployment/models/trained_edge_model.keras')
    
control_model.save('../deployment/models/trained_control_model.keras')


INFO:tensorflow:Assets written to: expt/assets


INFO:tensorflow:Assets written to: expt/assets


Saved artifact at 'expt'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 19), dtype=tf.float32, name='keras_tensor_76')
Output Type:
  List[TensorSpec(shape=(None, 32), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)]
Captures:
  13042796144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13042796320: TensorSpec(shape=(), dtype=tf.resource, name=None)
  12985234528: TensorSpec(shape=(), dtype=tf.resource, name=None)
  12985256768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13018981824: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13018983408: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1743706598.820083 2240442 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1743706598.821132 2240442 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-04-03 19:56:38.822749: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: ./expt
2025-04-03 19:56:38.823221: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-04-03 19:56:38.823226: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: ./expt
2025-04-03 19:56:38.834946: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-04-03 19:56:38.888195: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: ./expt
2025-04-03 19:56:38.893695: I tensorflow/cc/saved_model/loader.cc:466] SavedModel load for tags { serve }; Status: success: OK. Took 70946 microseconds.


In [14]:
# Check combined model's training loss and validation loss
training_loss = history.history['loss']
validation_loss = history.history['val_loss']

print("Training Loss:", training_loss)
print("Validation Loss:", validation_loss)

raw = [2, 0, 13.4, 0, 13.4, 13.4, 15.4, 100, 1022.1, 2, 8, 2, 220, 50, 64, 0.0, 4000, 3, 8]
datapoint = np.array([raw], dtype=np.float32)

# Predict using the combined model
output_data = combined_model.predict(datapoint)

# output_data will be a list of two arrays, one for each output (edge_output, control_output)
edge_output = output_data[0][0]  # First output: edge_output (this is the first array in the list)
control_output = output_data[1][0]  # Second output: control_output (this is the second array in the list)

print("Edge Output:", edge_output)  # This should give you the edge prediction
print("Control Output:", control_output)  # This should give you the control prediction


Training Loss: [0.25178205966949463, 0.23968347907066345, 0.24461109936237335, 0.24302606284618378, 0.23791706562042236, 0.24550817906856537, 0.243641659617424, 0.2393542379140854, 0.24527223408222198, 0.23985464870929718]
Validation Loss: [0.217668354511261, 0.2463953047990799, 0.2797223925590515, 0.21911336481571198, 0.27226147055625916, 0.26257964968681335, 0.270867258310318, 0.20820137858390808, 0.2942362129688263, 0.3463376760482788]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Edge Output: [0.06930117]
Control Output: [0.43953162]


# On the AI Edge Server

In [15]:
from ai_edge_litert.interpreter import Interpreter
interpreter = Interpreter(model_path='../deployment/models/edge_model.tflite')
signatures = interpreter.get_signature_list()
interpreter.allocate_tensors()

# FEED IN THE INPUT DATA
input_details = interpreter.get_input_details()
input_data = np.array([[0,0,8.7,0,8.6,8.5,11.1,99,1003.1,2,12,2,350,61,66,0.0,15000,45,8]], dtype=np.float32)
# Set input tensor
input_index = input_details[0]['index']
interpreter.set_tensor(input_index, input_data)
# GET THE OUTPUT DATA
interpreter.invoke()
output_details = interpreter.get_output_details()
rain_data = interpreter.get_tensor(output_details[0]['index'])
node_data = interpreter.get_tensor(output_details[1]['index'])
node_data = node_data[0] # Send this to control
prob_rain = rain_data[0]


# Determine if it's raining
RAIN_THRESHOLD = 0.5
prob_rain
node_data

array([415.3908  ,   0.      ,   0.      ,   0.      ,   0.      ,
         0.      ,   0.      ,   0.      ,   0.      ,   0.      ,
        31.634277,   0.      ,   0.      ,   0.      ,   0.      ,
         0.      ,   0.      ,   0.      ,   0.      ,   0.      ,
        66.00993 ,   0.      ,   0.      ,   0.      ,   0.      ,
         0.      , 146.64545 , 914.6798  ,   0.      ,   0.      ,
         0.      , 129.53992 ], dtype=float32)

# Control Model
## On control model

In [16]:
import numpy as np
import tensorflow as tf
control_model = tf.keras.models.load_model('../deployment/models/control_model.keras')

## Vars that need to be defined for the function 
# Recieve node_data, data_result
BATCH_SIZE = 1 
X_data = []  
Y_data = []  
data_result = 0.4
data_result = np.array([data_result])

edge_model_result = node_data
edge_model_result = np.reshape(edge_model_result, (-1, 32)) 

control_predictions = control_model.predict(edge_model_result)
print("Control predictions:", control_predictions)

# IF data result exists, add it and intermediate layer to batches
if data_result.size > 0: 
    # Add data_result to Y and beginning nodes to X
    X_data.append(edge_model_result) 
    Y_data.append(data_result) 
    if len(X_data) >= BATCH_SIZE:
        numpy_X_data = np.vstack(X_data)
        numpy_Y_data = np.vstack(Y_data)  

        # Clear lists for the next batch
        X_data.clear()
        Y_data.clear()
        
        dataset = tf.data.Dataset.from_tensor_slices((numpy_X_data, numpy_Y_data))
        dataset = dataset.batch(BATCH_SIZE)
        # Compile and train the control_model
        edge_model_result = tf.convert_to_tensor(edge_model_result, dtype=tf.float32)  # Convert to Tensor
        control_model.compile(optimizer='adam',
                              loss='mean_squared_error',
                              metrics=['mae'])

        #Get edge gradients
        with tf.GradientTape(persistent=True) as tape:
            tape.watch(edge_model_result)  # Track edge model output
            control_predictions = control_model(edge_model_result, training=True)
            loss = tf.keras.losses.MeanSquaredError()(data_result, control_predictions)
        edge_output_grad = tape.gradient(loss, edge_model_result)
        
        print("Training Model")
        # Below is how models are usually trained, we need to step down from the high level api of fit() and get slightly lower down in
        # tensorflow mush
        # history = control_model.fit(numpy_X_data, numpy_Y_data,
        #                             epochs=1, batch_size=BATCH_SIZE)
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

        for epoch in range(5):
            with tf.GradientTape() as tape: # This will track computations performed on tensors inside the block 
                for batch_X, batch_Y in dataset:
                    predictions = control_model(batch_X, training=True) # This is the forward pass 
                    loss = tf.keras.losses.MeanSquaredError()(batch_Y, predictions)
                control_gradients = tape.gradient(loss, control_model.trainable_variables) # gradients get calculated via calling tape.gradient 
                optimizer.apply_gradients(zip(control_gradients, control_model.trainable_variables))
            print(f"Epoch {epoch+1}, Loss: {loss.numpy()}")
            


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
Control predictions: [[0.12695009]]
Training Model
Epoch 1, Loss: 0.07455626130104065
Epoch 2, Loss: 0.06499973684549332
Epoch 3, Loss: 0.04834967106580734
Epoch 4, Loss: 0.02800249494612217
Epoch 5, Loss: 0.013295230455696583


2025-03-26 20:21:30.535635: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [17]:
!pip show tensorflow

Name: tensorflow
Version: 2.18.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /opt/anaconda3/lib/python3.12/site-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, requests, setuptools, six, tensorboard, termcolor, typing-extensions, wrapt
Required-by: 


## Split Learning on Edge v1

In [18]:
# # Load edge model
# edge_learning_model = tf.keras.models.load_model("../deployment/models/edge_model.keras")

# # Reinitialize the optimizer
# optimizer = tf.keras.optimizers.Adam()
# edge_learning_model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])

# for layer in edge_learning_model.layers:
#     layer.trainable=True
# edge_learning_model.get_layer("edge_output").trainable = False
# # Apply the received gradients
# grad_updates = [(gradients[10], edge_learning_model.trainable_variables[-2])]
# optimizer.apply_gradients(grad_updates)

In [19]:
# control_model.summary()
# edge_model.summary()

In [20]:
# edge_learning_model = tf.keras.models.load_model("../deployment/models/edge_model.keras")
# for layer in edge_learning_model.layers:
#     layer.trainable = True
# edge_learning_model.get_layer("edge_output").trainable=False

In [21]:
import tensorflow as tf
import numpy as np

# Load edge model
edge_learning_model = tf.keras.models.load_model("../deployment/models/edge_model.keras")
input_data = np.array([[0,0,8.7,0,8.6,8.5,11.1,99,1003.1,2,12,2,350,61,66,0.0,15000,45,8]], dtype=np.float32)
print(input_data)
edge_learning_model = tf.keras.models.load_model("../deployment/models/edge_model.keras")
for layer in edge_learning_model.layers:
    layer.trainable = True
edge_learning_model.get_layer("edge_output").trainable=False

# Reinitialize the optimizer
print(edge_learning_model.trainable_variables)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
edge_learning_model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])

# Assume edge_output_grad is received from the control machine
edge_output_grad = tf.convert_to_tensor(edge_output_grad, dtype=tf.float32)
edge_output_grad_temp=edge_output_grad
# Perform backpropagation on edge model
with tf.GradientTape() as tape:
    edge_output = edge_learning_model(input_data, training=True)  # Forward pass
tape.watch(edge_output)

# Compute edge model gradients
edge_model_grads = tape.gradient(edge_output, edge_learning_model.trainable_variables, output_gradients=edge_output_grad)

# Apply gradients to update edge model
optimizer.apply_gradients(zip(edge_model_grads, edge_learning_model.trainable_variables))

#Output layer for inference now needs to be retrained so just unfreeze it and refreeze other layers 


[[0.0000e+00 0.0000e+00 8.7000e+00 0.0000e+00 8.6000e+00 8.5000e+00
  1.1100e+01 9.9000e+01 1.0031e+03 2.0000e+00 1.2000e+01 2.0000e+00
  3.5000e+02 6.1000e+01 6.6000e+01 0.0000e+00 1.5000e+04 4.5000e+01
  8.0000e+00]]
[<Variable path=e1/kernel, shape=(19, 64), dtype=float32, value=[[-2.6573771e-01  4.1963282e-01 -6.9021044e+00 ... -6.3333832e-02
   1.8328018e-01 -2.3332907e-01]
 [-1.5872814e-01 -1.7659296e-01 -2.2302914e+00 ... -3.8706038e-02
   3.2268986e-01  1.5744790e-01]
 [ 6.2916189e-02 -1.6619353e-01  1.7655957e-01 ... -1.3904998e-01
   4.8682518e-02 -1.7637646e-01]
 ...
 [-3.3217636e-01 -1.3478510e-01 -5.7403995e-03 ...  1.5349241e-01
   2.3962392e-02  2.1852884e-03]
 [-2.1697301e-01 -3.8918626e-02 -1.8884225e-02 ...  8.7985210e-02
  -2.0280710e-01  1.3662296e-02]
 [-1.4357294e-01 -4.8413452e-02  3.4863114e-02 ...  1.3774540e-02
   6.0951140e-02  4.8851609e-02]]>, <Variable path=e1/bias, shape=(64,), dtype=float32, value=[-0.17048174 -0.20096089 -0.04104858 -0.02430137 -0.04245

<Variable path=adam/iteration, shape=(), dtype=int64, value=1>

# S1

In [22]:
## S+1

import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import Input, Model, layers
from sklearn.model_selection import train_test_split

# Load and preprocess data
RAIN_THRESHHOLD = 0.5
data = pd.read_csv('weather_data.csv', low_memory=False)
data = data.sample(frac=1, random_state=2).reset_index(drop=True)
data = data.replace('', float('nan')).dropna()
data = data.replace(' ', float('nan'))
data = data.dropna(subset=['wetb', 'vappr', 'rhum', 'vis'])
X = data.drop(columns=['rain','date'])
y = data['rain']
print(data.isna().sum())

# Ensure all data is numeric
X = X.astype(float)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape y to (num_samples, 1) for regression
y_train = y_train.values.reshape(-1, 1)
y_test = y_test.values.reshape(-1, 1)

# Define the model
inputs = Input(shape=(X_train.shape[1],))
x = layers.Dense(32, activation='relu', name='e1')(inputs)
edge_outputs = layers.Dense(1, activation='linear', name='edge_output')(x)
x = layers.Dense(64, activation='relu', name='controlmodel')(x)
x = layers.Dense(64, activation='relu', name='l1')(x)
x = layers.Dense(128, activation='relu', name='l2')(x)
x = layers.Dense(256, activation='relu', name='l3')(x)
x = layers.Dense(128, activation='relu', name='l4')(x)
x = layers.Dense(64, activation='relu', name='l5')(x)
x = layers.Dense(64, activation='relu', name='l6')(x)

control_outputs = layers.Dense(1, activation='linear', name='control_output')(x)

# Create and compile the combined model
combined_model = Model(inputs=inputs, outputs=[edge_outputs, control_outputs])
combined_model.compile(optimizer='adam',
                       loss={'edge_output': 'mean_squared_error',
                             'control_output': 'mean_squared_error'},
                       metrics={'edge_output': 'mae',
                                'control_output': 'mae'})

# Train the combined model
combined_model.fit(X_train, [y_train, y_train], 
                   epochs=15, batch_size=32, 
                   validation_data=(X_test, [y_test, y_test]))

# Define the edge_model
edge_model = Model(inputs=inputs, outputs=[combined_model.get_layer('e1').output, edge_outputs])

# x§This needs training 
# Set last as untrainable, then train last ones, this is not needed for contorl model becasue its last output is trained when entire model 
# gets trained
for layer in edge_model.layers:
    layer.trainable = False

edge_model.get_layer('edge_output').trainable= True

edge_model.compile(optimizer='adam',
                   loss={'e1': None, 'edge_output': 'mean_squared_error'},
                   metrics={'edge_output': 'mae'})

history = edge_model.fit(X_train, [y_train, y_train], 
               epochs=10, batch_size=32, 
               validation_data=(X_test, [y_test, y_test]))

# Define the control_model
control_model_input = Input(shape=(32,))  # Shape of the intermediate layer output
z = combined_model.get_layer('controlmodel')(control_model_input)
z = combined_model.get_layer('l1')(z)
z = combined_model.get_layer('l2')(z)
z = combined_model.get_layer('l3')(z)
z = combined_model.get_layer('l4')(z)
z = combined_model.get_layer('l5')(z)
z = combined_model.get_layer('l6')(z)


control_model_outputs = combined_model.get_layer('control_output')(z)
control_model = Model(inputs=control_model_input, outputs=control_model_outputs)

# Inspect the models
edge_model.summary()
control_model.summary()

edge_model.export("expt")
converter = tf.lite.TFLiteConverter.from_saved_model('./expt')
tflite_model = converter.convert()
with open('../deployment/models/edge_modelS1.tflite', 'wb') as f:
  f.write(tflite_model)

edge_model.save('../deployment/models/edge_modelS1.keras')
    
control_model.save('../deployment/models/control_modelS1.keras')



date     0
ind      0
rain     0
ind1     0
temp     0
ind2     0
wetb     0
dewpt    0
vappr    0
rhum     0
msl      0
ind3     0
wdsp     0
ind4     0
wddir    0
ww       0
w        0
sun      0
vis      0
clht     0
clamt    0
dtype: int64
Epoch 1/20
3184/3184 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - control_output_loss: 1527.8674 - control_output_mae: 4.3009 - edge_output_loss: 133042.8281 - edge_output_mae: 71.3840 - loss: 134570.6562 - val_control_output_loss: 3.8620 - val_control_output_mae: 1.6879 - val_edge_output_loss: 8.3348 - val_edge_output_mae: 2.4276 - val_loss: 12.1974
Epoch 2/20
3184/3184 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - control_output_loss: 1.9459 - control_output_mae: 0.4858 - edge_output_loss: 3.2221 - edge_output_mae: 1.2142 - loss: 5.1680 - val_control_output_loss: 0.8602 - val_control_output_mae: 0.7588 - val_edge_output_loss: 3.9321 - val_edge_output_mae: 1.7691 - val_loss: 4.7925
Epoch 3/20
3184/3184 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - control_output_loss: 0.3897 -

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 19)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ e1 (Dense)                      │ (None, 32)             │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ edge_output (Dense)             │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 741 (2.90 KB)

 Trainable params: 33 (132.00 B)

...

 Non-trainable params: 0 (0.00 B)

INFO:tensorflow:Assets written to: expt/assets


INFO:tensorflow:Assets written to: expt/assets


Saved artifact at 'expt'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 19), dtype=tf.float32, name='keras_tensor_84')
Output Type:
  List[TensorSpec(shape=(None, 32), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)]
Captures:
  6427065424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6427058128: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6427051600: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6427062736: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1743020583.272197 13507011 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1743020583.272699 13507011 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-03-26 20:23:03.273597: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: ./expt
2025-03-26 20:23:03.273804: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-03-26 20:23:03.273809: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: ./expt
2025-03-26 20:23:03.276271: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-03-26 20:23:03.289776: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: ./expt
2025-03-26 20:23:03.292691: I tensorflow/cc/saved_model/loader.cc:466] SavedModel load for tags { serve }; Status: success: OK. Took 19096 microseconds.


# S2
... same as original

# S3

In [23]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import Input, Model, layers
from sklearn.model_selection import train_test_split

# Load and preprocess data
RAIN_THRESHHOLD = 0.5
data = pd.read_csv('weather_data.csv', low_memory=False)
data = data.sample(frac=1, random_state=2).reset_index(drop=True)
data = data.replace('', float('nan')).dropna()
data = data.replace(' ', float('nan'))
data = data.dropna(subset=['wetb', 'vappr', 'rhum', 'vis'])
X = data.drop(columns=['rain','date'])
y = data['rain']
print(data.isna().sum())

# Ensure all data is numeric
X = X.astype(float)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape y to (num_samples, 1) for regression
y_train = y_train.values.reshape(-1, 1)
y_test = y_test.values.reshape(-1, 1)

# Define the model
inputs = Input(shape=(X_train.shape[1],))
x = layers.Dense(64, activation='relu', name='e1')(inputs)
z = layers.Dense(64, activation='relu', name='e2')(x)
x = layers.Dense(32, activation='relu', name='e3')(x)
edge_outputs = layers.Dense(1, activation='linear', name='edge_output')(x)
x = layers.Dense(64, activation='relu', name='controlmodel')(x)
x = layers.Dense(256, activation='relu', name='l1')(x)
x = layers.Dense(128, activation='relu', name='l2')(x)
x = layers.Dense(64, activation='relu', name='l3')(x)
x = layers.Dense(64, activation='relu', name='l4')(x)

control_outputs = layers.Dense(1, activation='linear', name='control_output')(x)

# Create and compile the combined model
combined_model = Model(inputs=inputs, outputs=[edge_outputs, control_outputs])
combined_model.compile(optimizer='adam',
                       loss={'edge_output': 'mean_squared_error',
                             'control_output': 'mean_squared_error'},
                       metrics={'edge_output': 'mae',
                                'control_output': 'mae'})

# Train the combined model
combined_model.fit(X_train, [y_train, y_train], 
                   epochs=20, batch_size=32, 
                   validation_data=(X_test, [y_test, y_test]))

# Define the edge_model
edge_model = Model(inputs=inputs, outputs=[combined_model.get_layer('e3').output, edge_outputs])

# x§This needs training 
# Set last as untrainable, then train last ones, this is not needed for contorl model becasue its last output is trained when entire model 
# gets trained
for layer in edge_model.layers:
    layer.trainable = False

edge_model.get_layer('edge_output').trainable= True

edge_model.compile(optimizer='adam',
                   loss={'e3': None, 'edge_output': 'mean_squared_error'},
                   metrics={'edge_output': 'mae'})

history = edge_model.fit(X_train, [y_train, y_train], 
               epochs=15, batch_size=32, 
               validation_data=(X_test, [y_test, y_test]))

# Define the control_model
control_model_input = Input(shape=(32,))  # Shape of the intermediate layer output
z = combined_model.get_layer('controlmodel')(control_model_input)
z = combined_model.get_layer('l1')(z)
z = combined_model.get_layer('l2')(z)
z = combined_model.get_layer('l3')(z)
z = combined_model.get_layer('l4')(z)

control_model_outputs = combined_model.get_layer('control_output')(z)
control_model = Model(inputs=control_model_input, outputs=control_model_outputs)

# Inspect the models
edge_model.summary()
control_model.summary()

edge_model.export("expt")
converter = tf.lite.TFLiteConverter.from_saved_model('./expt')
tflite_model = converter.convert()
with open('../deployment/models/edge_modelS3.tflite', 'wb') as f:
  f.write(tflite_model)

edge_model.save('../deployment/models/edge_modelS3.keras')
    
control_model.save('../deployment/models/control_modelS3.keras')



date     0
ind      0
rain     0
ind1     0
temp     0
ind2     0
wetb     0
dewpt    0
vappr    0
rhum     0
msl      0
ind3     0
wdsp     0
ind4     0
wddir    0
ww       0
w        0
sun      0
vis      0
clht     0
clamt    0
dtype: int64
Epoch 1/20
3184/3184 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - control_output_loss: 619.9954 - control_output_mae: 3.9663 - edge_output_loss: 7846.7236 - edge_output_mae: 19.3902 - loss: 8466.7246 - val_control_output_loss: 0.6813 - val_control_output_mae: 0.6981 - val_edge_output_loss: 185.2354 - val_edge_output_mae: 11.3749 - val_loss: 185.9262
Epoch 2/20
3184/3184 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - control_output_loss: 0.9753 - control_output_mae: 0.4970 - edge_output_loss: 589.5010 - edge_output_mae: 8.3955 - loss: 590.4766 - val_control_output_loss: 1.2968 - val_control_output_mae: 0.9513 - val_edge_output_loss: 22.5373 - val_edge_output_mae: 4.1327 - val_loss: 23.8357
Epoch 3/20
3184/3184 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - control_output_loss: 0.3

Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)      │ (None, 19)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ e1 (Dense)                      │ (None, 64)             │         1,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ e3 (Dense)                      │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ edge_output (Dense)             │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,461 (13.52 KB)

 Trainable params: 33 (132.00 B)

...

 Non-trainable params: 0 (0.00 B)

INFO:tensorflow:Assets written to: expt/assets


INFO:tensorflow:Assets written to: expt/assets


Saved artifact at 'expt'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 19), dtype=tf.float32, name='keras_tensor_104')
Output Type:
  List[TensorSpec(shape=(None, 32), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)]
Captures:
  6345019024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6427053712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6345022672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6441278480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6441278096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6441271376: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1743020666.984740 13507011 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1743020666.984928 13507011 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-03-26 20:24:26.985947: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: ./expt
2025-03-26 20:24:26.986236: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-03-26 20:24:26.986241: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: ./expt
2025-03-26 20:24:26.989280: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-03-26 20:24:27.005645: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: ./expt
2025-03-26 20:24:27.009526: I tensorflow/cc/saved_model/loader.cc:466] SavedModel load for tags { serve }; Status: success: OK. Took 23581 microseconds.


# S4

In [24]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import Input, Model, layers
from sklearn.model_selection import train_test_split

# Load and preprocess data
RAIN_THRESHHOLD = 0.5
data = pd.read_csv('weather_data.csv', low_memory=False)
data = data.sample(frac=1, random_state=2).reset_index(drop=True)
data = data.replace('', float('nan')).dropna()
data = data.replace(' ', float('nan'))
data = data.dropna(subset=['wetb', 'vappr', 'rhum', 'vis'])
X = data.drop(columns=['rain','date'])
y = data['rain']
print(data.isna().sum())

# Ensure all data is numeric
X = X.astype(float)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape y to (num_samples, 1) for regression
y_train = y_train.values.reshape(-1, 1)
y_test = y_test.values.reshape(-1, 1)

# Define the model
inputs = Input(shape=(X_train.shape[1],))
x = layers.Dense(64, activation='relu', name='e1')(inputs)
z = layers.Dense(64, activation='relu', name='e2')(x)
x = layers.Dense(64, activation='relu', name='e3')(x)
x = layers.Dense(32, activation='relu', name='e4')(x)
edge_outputs = layers.Dense(1, activation='linear', name='edge_output')(x)
x = layers.Dense(64, activation='relu', name='controlmodel')(x)
x = layers.Dense(256, activation='relu', name='l1')(x)
x = layers.Dense(128, activation='relu', name='l2')(x)
x = layers.Dense(64, activation='relu', name='l3')(x)

control_outputs = layers.Dense(1, activation='linear', name='control_output')(x)

# Create and compile the combined model
combined_model = Model(inputs=inputs, outputs=[edge_outputs, control_outputs])
combined_model.compile(optimizer='adam',
                       loss={'edge_output': 'mean_squared_error',
                             'control_output': 'mean_squared_error'},
                       metrics={'edge_output': 'mae',
                                'control_output': 'mae'})

# Train the combined model
combined_model.fit(X_train, [y_train, y_train], 
                   epochs=10, batch_size=32, 
                   validation_data=(X_test, [y_test, y_test]))

# Define the edge_model
edge_model = Model(inputs=inputs, outputs=[combined_model.get_layer('e4').output, edge_outputs])

# x§This needs training 
# Set last as untrainable, then train last ones, this is not needed for contorl model becasue its last output is trained when entire model 
# gets trained
for layer in edge_model.layers:
    layer.trainable = False

edge_model.get_layer('edge_output').trainable= True

edge_model.compile(optimizer='adam',
                   loss={'e4': None, 'edge_output': 'mean_squared_error'},
                   metrics={'edge_output': 'mae'})

history = edge_model.fit(X_train, [y_train, y_train], 
               epochs=15, batch_size=32, 
               validation_data=(X_test, [y_test, y_test]))

# Define the control_model
control_model_input = Input(shape=(32,))  # Shape of the intermediate layer output
z = combined_model.get_layer('controlmodel')(control_model_input)
z = combined_model.get_layer('l1')(z)
z = combined_model.get_layer('l2')(z)
z = combined_model.get_layer('l3')(z)

control_model_outputs = combined_model.get_layer('control_output')(z)
control_model = Model(inputs=control_model_input, outputs=control_model_outputs)

# Inspect the models
edge_model.summary()
control_model.summary()

edge_model.export("expt")
converter = tf.lite.TFLiteConverter.from_saved_model('./expt')
tflite_model = converter.convert()
with open('../deployment/models/edge_modelS4.tflite', 'wb') as f:
  f.write(tflite_model)

edge_model.save('../deployment/models/edge_modelS4.keras')
    
control_model.save('../deployment/models/control_modelS4.keras')



date     0
ind      0
rain     0
ind1     0
temp     0
ind2     0
wetb     0
dewpt    0
vappr    0
rhum     0
msl      0
ind3     0
wdsp     0
ind4     0
wddir    0
ww       0
w        0
sun      0
vis      0
clht     0
clamt    0
dtype: int64
Epoch 1/20
3184/3184 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - control_output_loss: 196.3886 - control_output_mae: 2.8870 - edge_output_loss: 4867.5928 - edge_output_mae: 17.4264 - loss: 5063.9800 - val_control_output_loss: 0.4330 - val_control_output_mae: 0.4465 - val_edge_output_loss: 35.4569 - val_edge_output_mae: 5.3040 - val_loss: 35.8899
Epoch 2/20
3184/3184 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - control_output_loss: 0.7699 - control_output_mae: 0.4697 - edge_output_loss: 137.8849 - edge_output_mae: 3.8237 - loss: 138.6546 - val_control_output_loss: 0.3448 - val_control_output_mae: 0.4064 - val_edge_output_loss: 1348.2474 - val_edge_output_mae: 31.2761 - val_loss: 1348.6633
Epoch 3/20
3184/3184 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - control_output_loss: 0

Model: "functional_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)      │ (None, 19)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ e1 (Dense)                      │ (None, 64)             │         1,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ e3 (Dense)                      │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ e4 (Dense)                      │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ edge_output (Dense)             │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,621 (29.77 KB)

 Trainable params: 33 (132.00 B)

...

 Non-trainable params: 0 (0.00 B)

INFO:tensorflow:Assets written to: expt/assets


INFO:tensorflow:Assets written to: expt/assets


Saved artifact at 'expt'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 19), dtype=tf.float32, name='keras_tensor_122')
Output Type:
  List[TensorSpec(shape=(None, 32), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)]
Captures:
  13085579536: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13085573584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13085586832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13085582992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13085583760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13085580304: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13085576848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13085577232: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1743020758.807434 13507011 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1743020758.807998 13507011 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-03-26 20:25:58.809778: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: ./expt
2025-03-26 20:25:58.810109: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-03-26 20:25:58.810115: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: ./expt
2025-03-26 20:25:58.814704: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-03-26 20:25:58.837347: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: ./expt
2025-03-26 20:25:58.842553: I tensorflow/cc/saved_model/loader.cc:466] SavedModel load for tags { serve }; Status: success: OK. Took 32780 microseconds.


## S5

In [25]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import Input, Model, layers
from sklearn.model_selection import train_test_split

# Load and preprocess data
RAIN_THRESHHOLD = 0.5
data = pd.read_csv('weather_data.csv', low_memory=False)
data = data.sample(frac=1, random_state=2).reset_index(drop=True)
data = data.replace('', float('nan')).dropna()
data = data.replace(' ', float('nan'))
data = data.dropna(subset=['wetb', 'vappr', 'rhum', 'vis'])
X = data.drop(columns=['rain','date'])
y = data['rain']
print(data.isna().sum())

# Ensure all data is numeric
X = X.astype(float)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape y to (num_samples, 1) for regression
y_train = y_train.values.reshape(-1, 1)
y_test = y_test.values.reshape(-1, 1)

# Define the model
inputs = Input(shape=(X_train.shape[1],))
x = layers.Dense(64, activation='relu', name='e1')(inputs)
z = layers.Dense(64, activation='relu', name='e2')(x)
x = layers.Dense(64, activation='relu', name='e3')(x)
x = layers.Dense(256, activation='relu', name='e4')(x)
x = layers.Dense(32, activation='relu', name='e5')(x)
edge_outputs = layers.Dense(1, activation='linear', name='edge_output')(x)
x = layers.Dense(64, activation='relu', name='controlmodel')(x)
x = layers.Dense(128, activation='relu', name='l1')(x)
x = layers.Dense(64, activation='relu', name='l2')(x)

control_outputs = layers.Dense(1, activation='linear', name='control_output')(x)

# Create and compile the combined model
combined_model = Model(inputs=inputs, outputs=[edge_outputs, control_outputs])
combined_model.compile(optimizer='adam',
                       loss={'edge_output': 'mean_squared_error',
                             'control_output': 'mean_squared_error'},
                       metrics={'edge_output': 'mae',
                                'control_output': 'mae'})

# Train the combined model
combined_model.fit(X_train, [y_train, y_train], 
                   epochs=15, batch_size=32, 
                   validation_data=(X_test, [y_test, y_test]))

# Define the edge_model
edge_model = Model(inputs=inputs, outputs=[combined_model.get_layer('e5').output, edge_outputs])

# x§This needs training 
# Set last as untrainable, then train last ones, this is not needed for contorl model becasue its last output is trained when entire model 
# gets trained
for layer in edge_model.layers:
    layer.trainable = False

edge_model.get_layer('edge_output').trainable= True

edge_model.compile(optimizer='adam',
                   loss={'e5': None, 'edge_output': 'mean_squared_error'},
                   metrics={'edge_output': 'mae'})

history = edge_model.fit(X_train, [y_train, y_train], 
               epochs=10, batch_size=32, 
               validation_data=(X_test, [y_test, y_test]))

# Define the control_model
control_model_input = Input(shape=(32,))  # Shape of the intermediate layer output
z = combined_model.get_layer('controlmodel')(control_model_input)
z = combined_model.get_layer('l1')(z)
z = combined_model.get_layer('l2')(z)

control_model_outputs = combined_model.get_layer('control_output')(z)
control_model = Model(inputs=control_model_input, outputs=control_model_outputs)

# Inspect the models
edge_model.summary()
control_model.summary()

edge_model.export("expt")
converter = tf.lite.TFLiteConverter.from_saved_model('./expt')
tflite_model = converter.convert()
with open('../deployment/models/edge_modelS5.tflite', 'wb') as f:
  f.write(tflite_model)

edge_model.save('../deployment/models/edge_modelS5.keras')
    
control_model.save('../deployment/models/control_modelS5.keras')



date     0
ind      0
rain     0
ind1     0
temp     0
ind2     0
wetb     0
dewpt    0
vappr    0
rhum     0
msl      0
ind3     0
wdsp     0
ind4     0
wddir    0
ww       0
w        0
sun      0
vis      0
clht     0
clamt    0
dtype: int64
Epoch 1/20
3184/3184 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - control_output_loss: 13693.0371 - control_output_mae: 13.5191 - edge_output_loss: 24105.9082 - edge_output_mae: 25.5536 - loss: 37798.9570 - val_control_output_loss: 0.3793 - val_control_output_mae: 0.4188 - val_edge_output_loss: 1.7812 - val_edge_output_mae: 1.1215 - val_loss: 2.1607
Epoch 2/20
3184/3184 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - control_output_loss: 0.3552 - control_output_mae: 0.3433 - edge_output_loss: 73.3040 - edge_output_mae: 3.6016 - loss: 73.6592 - val_control_output_loss: 0.3296 - val_control_output_mae: 0.3590 - val_edge_output_loss: 22.2616 - val_edge_output_mae: 4.1673 - val_loss: 22.5923
Epoch 3/20
3184/3184 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - control_output_loss: 0.629

Model: "functional_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_10 (InputLayer)     │ (None, 19)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ e1 (Dense)                      │ (None, 64)             │         1,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ e3 (Dense)                      │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ e4 (Dense)                      │ (None, 256)            │        16,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ e5 (Dense)                      │ (None, 32)             │         8,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ edge_output (Dense)             │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 30,405 (118.77 KB)

 Trainable params: 33 (132.00 B)

...

 Non-trainable params: 0 (0.00 B)

INFO:tensorflow:Assets written to: expt/assets


INFO:tensorflow:Assets written to: expt/assets


Saved artifact at 'expt'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 19), dtype=tf.float32, name='keras_tensor_139')
Output Type:
  List[TensorSpec(shape=(None, 32), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)]
Captures:
  4373901968: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6427054288: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13085589136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13048272400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13048270480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13085580880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6427058896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13048353936: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13048351248: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13048277968: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1743020837.807534 13507011 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1743020837.808102 13507011 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-03-26 20:27:17.810084: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: ./expt
2025-03-26 20:27:17.810471: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-03-26 20:27:17.810476: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: ./expt
2025-03-26 20:27:17.816310: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-03-26 20:27:17.841685: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: ./expt
2025-03-26 20:27:17.846796: I tensorflow/cc/saved_model/loader.cc:466] SavedModel load for tags { serve }; Status: success: OK. Took 36717 microseconds.


## S6

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import Input, Model, layers
from sklearn.model_selection import train_test_split

# Load and preprocess data
RAIN_THRESHHOLD = 0.5
data = pd.read_csv('weather_data.csv', low_memory=False)
data = data.sample(frac=1, random_state=2).reset_index(drop=True)
data = data.replace('', float('nan')).dropna()
data = data.replace(' ', float('nan'))
data = data.dropna(subset=['wetb', 'vappr', 'rhum', 'vis'])
X = data.drop(columns=['rain','date'])
y = data['rain']
print(data.isna().sum())

# Ensure all data is numeric
X = X.astype(float)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape y to (num_samples, 1) for regression
y_train = y_train.values.reshape(-1, 1)
y_test = y_test.values.reshape(-1, 1)

# Define the model
inputs = Input(shape=(X_train.shape[1],))
x = layers.Dense(64, activation='relu', name='e1')(inputs)
z = layers.Dense(64, activation='relu', name='e2')(x)
x = layers.Dense(64, activation='relu', name='e3')(x)
x = layers.Dense(256, activation='relu', name='e4')(x)
x = layers.Dense(128, activation='relu', name='e5')(x)
x = layers.Dense(32, activation='relu', name='e6')(x)
edge_outputs = layers.Dense(1, activation='linear', name='edge_output')(x)
x = layers.Dense(64, activation='relu', name='controlmodel')(x)
x = layers.Dense(64, activation='relu', name='l1')(x)

control_outputs = layers.Dense(1, activation='linear', name='control_output')(x)

# Create and compile the combined model
combined_model = Model(inputs=inputs, outputs=[edge_outputs, control_outputs])
combined_model.compile(optimizer='adam',
                       loss={'edge_output': 'mean_squared_error',
                             'control_output': 'mean_squared_error'},
                       metrics={'edge_output': 'mae',
                                'control_output': 'mae'})

# Train the combined model
combined_model.fit(X_train, [y_train, y_train], 
                   epochs=20, batch_size=32, 
                   validation_data=(X_test, [y_test, y_test]))

# Define the edge_model
edge_model = Model(inputs=inputs, outputs=[combined_model.get_layer('e6').output, edge_outputs])

# x§This needs training 
# Set last as untrainable, then train last ones, this is not needed for contorl model becasue its last output is trained when entire model 
# gets trained
for layer in edge_model.layers:
    layer.trainable = False

edge_model.get_layer('edge_output').trainable= True

edge_model.compile(optimizer='adam',
                   loss={'e6': None, 'edge_output': 'mean_squared_error'},
                   metrics={'edge_output': 'mae'})

history = edge_model.fit(X_train, [y_train, y_train], 
               epochs=10, batch_size=32, 
               validation_data=(X_test, [y_test, y_test]))

# Define the control_model
control_model_input = Input(shape=(32,))  # Shape of the intermediate layer output
z = combined_model.get_layer('controlmodel')(control_model_input)
z = combined_model.get_layer('l1')(z)

control_model_outputs = combined_model.get_layer('control_output')(z)
control_model = Model(inputs=control_model_input, outputs=control_model_outputs)

# Inspect the models
edge_model.summary()
control_model.summary()

edge_model.export("expt")
converter = tf.lite.TFLiteConverter.from_saved_model('./expt')
tflite_model = converter.convert()
with open('../deployment/models/edge_modelS6.tflite', 'wb') as f:
  f.write(tflite_model)

edge_model.save('../deployment/models/edge_modelS6.keras')
    
control_model.save('../deployment/models/control_modelS6.keras')



date     0
ind      0
rain     0
ind1     0
temp     0
ind2     0
wetb     0
dewpt    0
vappr    0
rhum     0
msl      0
ind3     0
wdsp     0
ind4     0
wddir    0
ww       0
w        0
sun      0
vis      0
clht     0
clamt    0
dtype: int64
Epoch 1/20
3184/3184 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - control_output_loss: 1369.9906 - control_output_mae: 7.5113 - edge_output_loss: 18949.2324 - edge_output_mae: 18.9685 - loss: 20319.2344 - val_control_output_loss: 3.4745 - val_control_output_mae: 1.6322 - val_edge_output_loss: 33.4103 - val_edge_output_mae: 4.9651 - val_loss: 36.8866
Epoch 2/20
3184/3184 ━━━━━━━━━━━━━━━━━━━━ 3s 982us/step - control_output_loss: 3.9416 - control_output_mae: 0.8238 - edge_output_loss: 116.7866 - edge_output_mae: 3.1298 - loss: 120.7283 - val_control_output_loss: 0.3621 - val_control_output_mae: 0.3552 - val_edge_output_loss: 0.3241 - val_edge_output_mae: 0.3112 - val_loss: 0.6863
Epoch 3/20
3184/3184 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - control_output_loss: 0.8